# Visualisation of semantic relations
### Using word embeddings

Author: [Camilo](https://github.com/camilocarvajalreyes)

**Objectives**: You should gain intuition on what kind of information gets
encoded in word representations and how to visualise it.


**To do**: You will code a **PCA visualisation** of semantic relations using
GloVe. They can base their algorithm in an example given with
countries and capitals.

## Instructions
1. Download the vectors, install packages and test if everything is okay (if you haven't done so yet)
2. Load the packages and import vectors by running cells in section 0
3. Take a look at the code below in which we visualise the GloVe embedding space
4. Based on the given code, make your own PCA visualisation of semantic relations.
5. Don't forget to ask if you have any question

## Section 0
Loading packages and importing pre-trained vectors. **Run all cells in this section**

In [ ]:
import matplotlib.pyplot as plt # visualisation
plt.rcParams["figure.figsize"] = (10,6) # setting figure size
import numpy as np # for operations with vectors
import csv # opening csv files
import pickle # opening pickled dictionary with vectors
from sklearn.decomposition import PCA # dimensionality reduction

**GloVe pre-trained vectors**


First we import a pickled dictionary with GloVe pre-trained vectors. We only import the vectors with the words we'll use in this notebook. For full vocabularies see [Stanford-GloVe](https://nlp.stanford.edu/projects/glove/).

Pennington, J., Socher, R., & Manning, C. D. (2014, October). [Glove: Global vectors for word representation](https://www.aclweb.org/anthology/D14-1162.pdf). In Proceedings of the 2014 conference on empirical methods in natural language processing (EMNLP) (pp. 1532-1543).

In [ ]:
vectors_path = 'data/glove_subset.pickle' # change path if necessary
with open(vectors_path, 'rb') as handle:
    vecs = pickle.load(handle)
    
# example
print('vector length = '+str(len(vecs['france'])))
vecs['france'][:5] # first five values

## Section 1: Visualising country-capital relation
Below there's an example of semantic relation visualisation.

We'll store capitals and countries in a dictionary

In [ ]:
capital_country={}
with open('data/country_list.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    for row in readCSV:
        country=row[0]
        country=country.lower().replace(' ','_') # remove upper case and add _ for n-grams
        capital=row[1]
        capital=capital.lower().replace(' ','_')
        try:
            vecs[capital]
            vecs[country]
            capital_country[country] = capital
        except Exception:
            pass
        
all_words = [word for word in capital_country.keys()]+[word for word in capital_country.values()]

**Dimensionality reduction using [PCA](https://scikit-learn.org/stable/modules/decomposition.html#pca)**

In [ ]:
all_vectors = [vecs[word] for word in all_words]

pca = PCA(n_components=2)
pca_vecs = pca.fit_transform(all_vectors)

Visualisation using matplotlib

In [ ]:
colours = ['tab:blue']*len(capital_country)+['tab:orange']*len(capital_country)
x_vecs, y_vecs = [v[0] for v in pca_vecs], [v[1] for v in pca_vecs]
plt.scatter(x_vecs, y_vecs, c=colours)
plt.title('Visualisation of GloVe - countries and capitals')

labels = ['chile','santiago','france','paris','canada','ottawa','tunisia','tunis','indonesia','jakarta']
for element in labels:
    vec = pca.transform(vecs[element].reshape(1,300))
    plt.text(vec[:,0], vec[:,1], element, fontweight='bold')

Here we have performed a PCA reduction on ~120 Capital-Country pairs, and we have have tagged some fo them.

We can visually experiment with the embedding space: let's consider the case in which we want to know which one is the capital of Chile. If we know that Paris is the capital of France, then we could try adding the vector representing the path between capitals and countries:

In [ ]:
countries_capitals_subset = ['france','paris','chile','santiago']
pca_words = pca.transform([vecs[word] for word in countries_capitals_subset])

x_vecs, y_vecs = [v[0] for v in pca_words], [v[1] for v in pca_words]
plt.scatter(x_vecs, y_vecs)
plt.title('Visualisation of GloVe - countries and capitals')
for element in countries_capitals_subset:
    vec = pca.transform(vecs[element].reshape(1,300))
    plt.text(vec[:,0], vec[:,1], element, fontweight='bold')
    
# drawing arrows
vec_france, vec_paris = pca_words[0], pca_words[1]
arrow_france = vec_paris-vec_france
plt.arrow(vec_france[0],vec_france[1],arrow_france[0],arrow_france[1])
# now we approximate for Chile
vec_chile, vec_stgo = pca_words[2], pca_words[3]
# theoretical arrow
plt.arrow(vec_chile[0],vec_chile[1],arrow_france[0],arrow_france[1],color='red')
# actual arrow
arrow_chile = vec_stgo-vec_chile
plt.arrow(vec_chile[0],vec_chile[1],arrow_chile[0],arrow_chile[1],linestyle=':')

## Section 2: Visualising Semantic Relations

You're given a dataset of **adjectives** with their respective **comparatives and superlatives**.

For instance: high - higher - highest

In [ ]:
words = []
with open('data/adjectives.csv') as csvfile: # Change path if necessary
    readCSV = csv.reader(csvfile)
    for row in readCSV:
        #print(len(row[0]))
        words.append(row[0])
print(words[:6])

adj_compa = {}
adj_supe = {}
#print(int(len(words)/3))
for i in np.arange(int(len(words)/3)):
    adj = words[3*i-2]
    compa = words[3*i-3]
    supe = words[3*i-1]
    try:
        vecs[adj],vecs[compa],vecs[supe]
        adj_compa[adj] = compa
        adj_supe[adj] = supe
    except Exception:
        pass

all_words = [word for word in adj_compa.keys()]+[word for word in adj_compa.values()] + [word for word in adj_supe.values()]

#print(all_words)

### To Do:
1. Perform a PCA reduction on the set.
2. Visualise this space
3. Do you notice that GloVe encodes the relationship between the three kinds of word?

**Dimensionality reduction using PCA**

In [ ]:
#Python 3.7.9

vector_words = [vecs[word] for word in all_words]

pca_adj = PCA(n_components = 3)
pca_vecs = pca_adj.fit_transform(vector_words)
#vect_projected = pca_adj.transform(words)

Visualisation using matplotlib

In [ ]:
len_plot = len(adj_compa)
colours = ['tab:blue']*len_plot+['tab:orange']*len_plot+['tab:green']*len_plot
x_vecs, y_vecs, z_vecs = [v[0] for v in pca_vecs], [v[1] for v in pca_vecs],[v[2] for v in pca_vecs]
plt.scatter(x_vecs, y_vecs,z_vecs, c=colours, marker='o')
plt.title('Visualisation of adjectives space')

labels = ['rougher', 'rough', 'roughest', 'meaner', 'mean', 'meanest']
for element in labels:
    vec = pca_adj.transform(vecs[element].reshape(1,300))
    plt.text(vec[:,0], vec[:,1], element, fontweight='bold')

**Bonus task**
1. Take a subset of the words and draw arrows between the three different pairs (adj-com, adj-sup, com-sup), a color for each pair. Is it consistent? 
2. Can you think of an algorithm to automatically detect whether it is a base adjective, comparative or superlative?

In [ ]:
### to do ###
adj_compa_subset = [ 'rough','rougher', 'mean', 'meaner' ]
adj_supe_subset =['rough', 'roughest',  'mean', 'meanest']
pca_compa = pca_adj.transform([vecs[word] for word in adj_compa_subset])
pca_supe = pca_adj.transform([vecs[word] for word in adj_supe_subset])

x_vecs, y_vecs = [v[0] for v in pca_compa], [v[1] for v in pca_compa]
plt.scatter(x_vecs, y_vecs)
plt.title('Visualisation of adj-comparative')
for element in adj_compa_subset:
    vec = pca_adj.transform(vecs[element].reshape(1,300))
    plt.text(vec[:,0], vec[:,1], element, fontweight='bold')
    
# drawing arrows
vec_rough, vec_rougher = pca_compa[0], pca_compa[1]
arrow_rough = vec_rougher-vec_rough
plt.arrow(vec_rough[0],vec_rough[1],arrow_rough[0],arrow_rough[1])
# now we approximate for Chile
vec_mean, vec_meaner= pca_compa[2], pca_compa[3]
# theoretical arrow
plt.arrow(vec_mean[0],vec_mean[1],arrow_rough[0],arrow_rough[1],color='red')
# actual arrow
arrow_mean = vec_meaner-vec_mean
plt.arrow(vec_mean[0],vec_mean[1],arrow_mean[0],arrow_mean[1],linestyle=':')
